In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

pd.set_option("display.max_rows", 100)

LARGE_FIGSIZE = (12, 8)

## Version Check

A quick check to verify that we are on at least version 1.2 of SQLAlchemy:

In [42]:
import sqlalchemy
sqlalchemy.__version__

'1.1.5'

## Connecting

For this exploration I will use a local development instance of the logware3 database. To connect we use create_engine():

In [6]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///:memory:', echo=True)
engine

Engine(sqlite:///:memory:)

The **echo** flag is a shortcut to setting up SQLAlchemy logging, which is accomplished via Python’s standard **logging** module. With it enabled, we’ll see all the generated SQL produced. If you are working through this tutorial and want less output generated, set it to **False**. This tutorial will format the SQL behind a popup window so it doesn’t get in our way; just click the “SQL” links to see what’s being generated.

The return value of **create_engine()** is an instance of **Engine**, and it represents the core interface to the database, adapted through a dialect that handles the details of the database and **DBAPI** in use. In this case the **PostgreSQL** dialect will interpret instructions to the Python built-in sqlite3 module.

The first time a method like **Engine.execute()** or **Engine.connect()** is called, the **Engine** establishes a real **DBAPI** connection to the database, which is then used to emit the SQL.

In [14]:
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey
metadata = MetaData()
users = Table('users', metadata,
    Column('id', Integer, primary_key=True),
    Column('name', String),
    Column('fullname', String),
)

In [15]:
addresses = Table('addresses', metadata,
  Column('id', Integer, primary_key=True),
  Column('user_id', None, ForeignKey('users.id')),
  Column('email_address', String, nullable=False)
)

In [16]:
metadata.create_all(engine)

2017-03-30 21:23:33,721 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2017-03-30 21:23:33,723 INFO sqlalchemy.engine.base.Engine ()
2017-03-30 21:23:33,725 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("addresses")
2017-03-30 21:23:33,726 INFO sqlalchemy.engine.base.Engine ()
2017-03-30 21:23:33,729 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2017-03-30 21:23:33,729 INFO sqlalchemy.engine.base.Engine ()
2017-03-30 21:23:33,731 INFO sqlalchemy.engine.base.Engine COMMIT
2017-03-30 21:23:33,733 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE addresses (
	id INTEGER NOT NULL, 
	user_id INTEGER, 
	email_address VARCHAR NOT NULL, 
	PRIMARY KEY (id), 
	FOREIGN KEY(user_id) REFERENCES users (id)
)


2017-03-30 21:23:33,734 INFO sqlalchemy.engine.base.Engine ()
2017-03-30 21:23:33,736 INFO sqlalchemy.engine.base.Engine COMMIT


In [17]:
ins = users.insert()

In [18]:
str(ins)

'INSERT INTO users (id, name, fullname) VALUES (:id, :name, :fullname)'

In [19]:
ins = users.insert().values(name='jack', fullname='Jack Jones')
str(ins)

'INSERT INTO users (name, fullname) VALUES (:name, :fullname)'

In [20]:
ins.compile().params

{'fullname': 'Jack Jones', 'name': 'jack'}

In [21]:
conn = engine.connect()
conn

In [22]:
result = conn.execute(ins)

2017-03-30 21:23:57,282 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname) VALUES (?, ?)
2017-03-30 21:23:57,285 INFO sqlalchemy.engine.base.Engine ('jack', 'Jack Jones')
2017-03-30 21:23:57,288 INFO sqlalchemy.engine.base.Engine COMMIT


In [23]:
ins = users.insert()
conn.execute(ins, id=2, name='wendy', fullname='Wendy Williams')

2017-03-30 21:29:50,523 INFO sqlalchemy.engine.base.Engine INSERT INTO users (id, name, fullname) VALUES (?, ?, ?)
2017-03-30 21:29:50,526 INFO sqlalchemy.engine.base.Engine (2, 'wendy', 'Wendy Williams')
2017-03-30 21:29:50,528 INFO sqlalchemy.engine.base.Engine COMMIT


In [24]:
conn.execute(addresses.insert(), [
   {'user_id': 1, 'email_address' : 'jack@yahoo.com'},
   {'user_id': 1, 'email_address' : 'jack@msn.com'},
   {'user_id': 2, 'email_address' : 'www@www.org'},
   {'user_id': 2, 'email_address' : 'wendy@aol.com'},
])

2017-03-30 21:31:44,192 INFO sqlalchemy.engine.base.Engine INSERT INTO addresses (user_id, email_address) VALUES (?, ?)
2017-03-30 21:31:44,194 INFO sqlalchemy.engine.base.Engine ((1, 'jack@yahoo.com'), (1, 'jack@msn.com'), (2, 'www@www.org'), (2, 'wendy@aol.com'))
2017-03-30 21:31:44,196 INFO sqlalchemy.engine.base.Engine COMMIT


In [25]:
from sqlalchemy.sql import select
s = select([users])
result = conn.execute(s)

2017-03-30 21:33:55,371 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users
2017-03-30 21:33:55,375 INFO sqlalchemy.engine.base.Engine ()


In [27]:
for row in result:
    print(row)

(1, 'jack', 'Jack Jones')
(2, 'wendy', 'Wendy Williams')


In [29]:
for row in conn.execute(s):
    print('name:', row[users.c.name], '; fullname:', row[users.c.fullname])

2017-03-30 21:46:30,049 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users
2017-03-30 21:46:30,051 INFO sqlalchemy.engine.base.Engine ()
name: jack ; fullname: Jack Jones
name: wendy ; fullname: Wendy Williams


In [30]:
result.close()

In [31]:
s = select([users.c.name, users.c.fullname])
result = conn.execute(s)
for row in result:
    print(row)

2017-03-30 21:50:55,129 INFO sqlalchemy.engine.base.Engine SELECT users.name, users.fullname 
FROM users
2017-03-30 21:50:55,130 INFO sqlalchemy.engine.base.Engine ()
('jack', 'Jack Jones')
('wendy', 'Wendy Williams')


In [32]:
for row in conn.execute(select([users, addresses])):
    print(row)

2017-03-30 21:55:42,083 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname, addresses.id, addresses.user_id, addresses.email_address 
FROM users, addresses
2017-03-30 21:55:42,084 INFO sqlalchemy.engine.base.Engine ()
(1, 'jack', 'Jack Jones', 1, 1, 'jack@yahoo.com')
(1, 'jack', 'Jack Jones', 2, 1, 'jack@msn.com')
(1, 'jack', 'Jack Jones', 3, 2, 'www@www.org')
(1, 'jack', 'Jack Jones', 4, 2, 'wendy@aol.com')
(2, 'wendy', 'Wendy Williams', 1, 1, 'jack@yahoo.com')
(2, 'wendy', 'Wendy Williams', 2, 1, 'jack@msn.com')
(2, 'wendy', 'Wendy Williams', 3, 2, 'www@www.org')
(2, 'wendy', 'Wendy Williams', 4, 2, 'wendy@aol.com')


In [35]:
s = select([users, addresses]).where(users.c.id == addresses.c.user_id)
for row in conn.execute(s):
    print(row)

2017-03-30 22:10:02,941 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname, addresses.id, addresses.user_id, addresses.email_address 
FROM users, addresses 
WHERE users.id = addresses.user_id
2017-03-30 22:10:02,944 INFO sqlalchemy.engine.base.Engine ()
(1, 'jack', 'Jack Jones', 1, 1, 'jack@yahoo.com')
(1, 'jack', 'Jack Jones', 2, 1, 'jack@msn.com')
(2, 'wendy', 'Wendy Williams', 3, 2, 'www@www.org')
(2, 'wendy', 'Wendy Williams', 4, 2, 'wendy@aol.com')


In [38]:
str(users.c.id == addresses.c.user_id)

'users.id = addresses.user_id'

In [39]:
from sqlalchemy.sql import and_, or_, not_

In [40]:
print(and_(
        users.c.name.like('j%'),
        users.c.id == addresses.c.user_id,
        or_(
             addresses.c.email_address == 'wendy@aol.com',
             addresses.c.email_address == 'jack@yahoo.com'
        ),
        not_(users.c.id > 5)
      )
 )

users.name LIKE :name_1 AND users.id = addresses.user_id AND (addresses.email_address = :email_address_1 OR addresses.email_address = :email_address_2) AND users.id <= :id_1


In [47]:
create_engine(sqlalchemy.engine.url.URL('postgresql+psycopg2', username='postgres', password=None, host='localhost', port=5432, database='dev_logware3'))

Engine(postgresql+psycopg2://postgres@localhost:5432/dev_logware3)

In [51]:
conn.info

{}